In [2]:
# importeer de benodigde bibliotheken, dan gaat het later sneller 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from kaggle.api.kaggle_api_extended import KaggleApi
import os
import zipfile
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
from tqdm import tqdm_notebook, tqdm

import random

# interactieve modus inschakelen en output verbreden
%matplotlib inline
pd.set_option('display.width', 800)

# remove future warnings
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings("ignore")

seed = 42
random.seed(seed)
np.random.seed(seed)


In [3]:
from kaggle.api.kaggle_api_extended import KaggleApi
import os
import zipfile

api = KaggleApi()
api.authenticate()

# path settings
from pathlib import Path
p = Path()
download_path = p / 'data'
output_path = p / 'output'

# download the data is niet gelukt, dus maar manueel gedaan

# Lees het bestand in
print(p)
for dirname, _, filenames in os.walk(p):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        if 'train' in filename:
            train_set = os.path.join(dirname, filename)
            break

train_raw = pd.read_csv(train_set, index_col='ID')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')
data = train_raw.copy()

.
.\london-house-price-prediction-advanced-techniques.zip
.\PLHP_1_CatBoostRegressor.ipynb
.\PLHP_2_Eval_models copy.ipynb
.\PLHP_2_Eval_models.ipynb
.\PLHP_3_Pipelines.ipynb
.\test_kaggle_DL.ipynb
.\catboost_info\catboost_training.json
.\catboost_info\learn\events.out.tfevents
.\catboost_info\test\events.out.tfevents
.\data\sample_submission.csv
.\data\test.csv
.\data\train.csv
.\images\model_comparison.png
.\output\profiler.html
.\output\profiler_raw.html
.\output\submission.csv
.\output\submission_cbr_log.csv


In [3]:
# Functie om de output van de gemaakte plots te bewaren

images_path = p / 'images'

# Create the images directory if it does not exist
if not images_path.exists():
	images_path.mkdir(parents=True, exist_ok=True)

def save_fig(fig_name, tight_layout=True, fig_extension="png", resolution=300):
    path = images_path / f"{fig_name}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


In [5]:
# Functie om NaN-waarden te behandelen

# Lijst om kolommen met >50% missende waarden op te slaan
cols_to_drop = []

def fill_missing_values(df):
    for col, missing_count in df.isnull().sum().items():
        missing_ratio = missing_count / len(df)
        
        if 0 < missing_ratio <= 0.5:
            if df[col].dtype == object:  # Categorische kolommen
                df[col] = df[col].fillna(df[col].mode()[0])
            else:  # Numerieke kolommen
                df[col] = df[col].fillna(df[col].median())
        
        elif missing_ratio > 0.5:
            cols_to_drop.append(col)

# Kolommen met te veel missende waarden verwijderen
data = data.drop(columns=cols_to_drop)

del cols_to_drop  # Opschoonactie

In [7]:
# Missende waarden invullen in test- en train-data en kilommen met te veel lege waarden droppen
# alleen uitvoeren indien nog lege waarden
if data.isnull().sum().sum() > 0:
    fill_missing_values(test)
    fill_missing_values(data)
    print(f"De lege waarden werdenn opgevuld en de kolommen met te veel missende waarden zijn verwijderd")
else:
    print("Er zijn geen missende waarden")


Er zijn geen missende waarden


In [6]:
# print de kolommen die niet numeriek zijn
non_numeric_cols = data.select_dtypes(include='object').columns
non_numeric_cols
#print het aantal unieke waarden per kolom
for col in non_numeric_cols:
    print(f"{col}: {data[col].nunique()} unieke waarden")

fullAddress: 118345 unieke waarden
postcode: 45064 unieke waarden
country: 1 unieke waarden
outcode: 167 unieke waarden
tenure: 4 unieke waarden
propertyType: 19 unieke waarden
currentEnergyRating: 7 unieke waarden


Stap voor stap opkuisen

In [ ]:
# indien geen kolom pstcode aanwezig is, dan deze toevoegen
if 'postcode' not in data.columns and 'fullAddress' in data.columns:
    # Extracteer postcode uit adres (bijvoorbeeld 'SE5 8AB' uit 'Flat 6, 7 De Crespigny Park, London, SE5 8AB')
    data['postcode'] = data['fullAddress'].str.extract(r'(\w{1,2}\d{1,2} \d{1,2}[A-Z]{1,2})', expand=False)
    # Verwijder de volledige adreskolom
    data.drop(columns='fullAddress', inplace=True)

# Omzetten de kolom 'outcode' via labelencoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for col in ['outcode', 'tenure', 'propertyType', 'currentEnergyRating']:
    data[col] = label_encoder.fit_transform(data[col])

# slechts één waarde voor country, dus ook weg ermee
if 'country' in data.columns: data.drop(columns='country', inplace=True)

# de Price kolom is niet normaal verdeeld, dus logaritmisch transformeren
data.price = np.log(data.price)

# kolommen met objecten verwijderen
data = data.select_dtypes(exclude='object')



In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 266325 entries, 0 to 266324
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   outcode              266325 non-null  int32  
 1   latitude             266325 non-null  float64
 2   longitude            266325 non-null  float64
 3   bathrooms            266325 non-null  float64
 4   bedrooms             266325 non-null  float64
 5   floorAreaSqM         266325 non-null  float64
 6   livingRooms          266325 non-null  float64
 7   tenure               266325 non-null  int32  
 8   propertyType         266325 non-null  int32  
 9   currentEnergyRating  266325 non-null  int32  
 10  sale_month           266325 non-null  int64  
 11  sale_year            266325 non-null  int64  
 12  price                266325 non-null  int64  
dtypes: float64(6), int32(4), int64(3)
memory usage: 24.4 MB


In [9]:
# TRain-Test-Split, let hier op dat we met een validatie set werken (en geen test set)

X =  data.drop(columns=['price'])
y =  data['price']
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
    test_size=0.2,
    shuffle=True,
    random_state=seed
)

In [ ]:

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb

def evaluate_models(X_train, y_train, X_valid, y_valid):
    # Dictionary voor de modellen en hun namen
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest Regressor': RandomForestRegressor(n_estimators=100, random_state=seed),
        'Gradient Boosting Regressor': GradientBoostingRegressor(n_estimators=100, random_state=seed),
        'XGBoost Regressor': xgb.XGBRegressor(n_estimators=100, random_state=seed),
        # 'Support Vector Regressor': SVR(),
        # 'CatBoost Regressor': CatBoostRegressor(
        #     use_best_model=True,
        #     random_state=seed,
        #     verbose=0  # Zet verbose op 0 om minder output te krijgen
        # )
    }
    
    # Opslaan van de resultaten
    results = {}
    
    # Itereer over de modellen, train en evalueer
    for name, model in models.items():
        # Train het model
        model.fit(X_train, y_train)
        
        # Voorspel de validatieset
        y_pred = model.predict(X_valid)
        
        # Bereken de evaluatie-metrics
        mae = mean_absolute_error(y_valid, y_pred)
        mse = mean_squared_error(y_valid, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_valid, y_pred)
        
        # Voeg de resultaten toe aan de dictionary
        results[name] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R²': r2}
    # nog een extra model toevoegen
    
    cbr = CatBoostRegressor(
        use_best_model=True,
        random_state=seed,
        verbose=200  # Zet verbose op 0 om minder output te krijgen
    )
    cbr.fit(X_train,
            y_train,
            eval_set=(X_valid, y_valid),
            cat_features=['postcode', 'outcode', 'tenure', 'propertyType', 'currentEnergyRating'],
            text_features=['fullAddress'],
            early_stopping_rounds=128
            )
    y_pred_log = cbr.predict(test[X_train.columns.tolist()])
    y_pred = np.round(np.expm1(y_pred_log),0).astype(int)

    # Bereken de evaluatie-metrics
    mae = mean_absolute_error(y_valid, y_pred)
    mse = mean_squared_error(y_valid, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_valid, y_pred)
    
    # Voeg de resultaten toe aan de dictionary
    results[name] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R²': r2}
    
    return results


In [17]:
def plot_comparison(results):
    # Haal de waarden voor MAE (of een andere metric) uit de results dictionary
    models = list(results.keys())
    maes = [results[model]['MAE'] for model in models]
    mse = [results[model]['MSE'] for model in models]
    rmse = [results[model]['RMSE'] for model in models]
    r2 = [results[model]['R²'] for model in models]
    
    # Plot de resultaten
    fig, ax = plt.subplots(2, 2, figsize=(14, 10))

    # MAE plot
    ax[0, 0].barh(models, maes, color='skyblue')
    ax[0, 0].set_title('Mean Absolute Error (MAE)')
    
    # MSE plot
    ax[0, 1].barh(models, mse, color='lightgreen')
    ax[0, 1].set_title('Mean Squared Error (MSE)')
    
    # RMSE plot
    ax[1, 0].barh(models, rmse, color='salmon')
    ax[1, 0].set_title('Root Mean Squared Error (RMSE)')
    
    # R² plot
    ax[1, 1].barh(models, r2, color='lightcoral')
    ax[1, 1].set_title('R²')

    plt.tight_layout()
    save_fig('model_comparison')
    plt.show()

# Voer de modellen uit en evalueer ze
results = evaluate_models(X_train, y_train, X_valid, y_valid)

# Visualiseer de vergelijking
plot_comparison(results)


KeyboardInterrupt: 